In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import glob as gb
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/emotions_classification/archive (1).zip','r') as zip_ref:
  zip_ref.extractall('/content/drive/MyDrive/emotions_classification2')

In [ ]:
TRAIN_DIR='/content/drive/MyDrive/emotions_classification2/eINTERFACE_2021_Image/train'
TEST_DIR='/content/drive/MyDrive/emotions_classification2/eINTERFACE_2021_Image/test'
BATCH_SIZE=64

Will see how many categories and images present

In [ ]:
for folder in os.listdir(TRAIN_DIR):
    files = gb.glob(pathname= str(TRAIN_DIR+ '/'+ folder + '/*.jpg'))
    print(f'For training data, found {len(files)} in folder {folder}')

For training data, found 1896 in folder Anger
For training data, found 1891 in folder Disgust
For training data, found 1922 in folder Fear
For training data, found 1922 in folder Happiness
For training data, found 1922 in folder Sadness
For training data, found 1922 in folder Surprise


In [ ]:
for folder in os.listdir(TEST_DIR):
    files = gb.glob(pathname= str(TEST_DIR+ '/'+ folder + '/*.jpg'))
    print(f'For testing data, found {len(files)} in folder {folder}')

For testing data, found 237 in folder Anger
For testing data, found 237 in folder Disgust
For testing data, found 241 in folder Fear
For testing data, found 241 in folder Happiness
For testing data, found 241 in folder Sadness
For testing data, found 241 in folder Surprise


In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def view_random_image(target_dir, target_class):
    # We will view images from here
    target_folder = target_dir + target_class
    
    # Get a random image path
    random_image = random.sample(od.listdir(target_folder), 1)
    
    # read in the image and plot it using matplolib
    img = mpimg.imread(target_folder+'/'+random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis('off')
    print(f"Image shape {img.shape}")
    
    return img

In [ ]:
class_names = ['Anger','Disgust','Fear','Happiness','Sadness','Surprise']

Preparing data for training

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                                 target_size = (128, 128),
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(TEST_DIR,
                                            target_size = (128, 128),
                                            batch_size = BATCH_SIZE,
                                            class_mode = 'categorical')

Found 11475 images belonging to 6 classes.
Found 1438 images belonging to 6 classes.


Basic model building (CNN Classifier)

In [ ]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(16, (3, 3), input_shape = (128, 128, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))



# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))

classifier.add(Dense(units = 6, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
## model summary
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 28800)             0         
                                                                 
 dense (Dense)               (None, 128)               3

In [ ]:

history = classifier.fit(training_set,
                         epochs = 50,
                         validation_data = test_set)


classifier.save('model1.h5')  # creates a HDF5 file 'my_model.h5'


Epoch 1/50
180/180 [==============================] - 270s 1s/step - loss: 1.8012 - accuracy: 0.1957 - val_loss: 1.7311 - val_accuracy: 0.2246
Epoch 2/50
180/180 [==============================] - 236s 1s/step - loss: 1.6209 - accuracy: 0.3285 - val_loss: 1.5125 - val_accuracy: 0.3769
Epoch 3/50
180/180 [==============================] - 236s 1s/step - loss: 1.4755 - accuracy: 0.4030 - val_loss: 1.4125 - val_accuracy: 0.4117
Epoch 4/50
180/180 [==============================] - 231s 1s/step - loss: 1.3597 - accuracy: 0.4448 - val_loss: 1.2608 - val_accuracy: 0.4805
Epoch 5/50
180/180 [==============================] - 227s 1s/step - loss: 1.2650 - accuracy: 0.4942 - val_loss: 1.1691 - val_accuracy: 0.5313
Epoch 6/50
180/180 [==============================] - 234s 1s/step - loss: 1.1788 - accuracy: 0.5332 - val_loss: 1.1200 - val_accuracy: 0.5494
Epoch 7/50
180/180 [==============================] - 233s 1s/step - loss: 1.1059 - accuracy: 0.5606 - val_loss: 1.0543 - val_accuracy: 0.5848

Evaluating the model

In [ ]:
classifier.evaluate(test_set)

23/23 [==============================] - 20s 889ms/step - loss: 0.6006 - accuracy: 0.7803


[0.600553035736084, 0.7802503705024719]

In [ ]:
model_path = "model1.h5"
loaded_model = keras.models.load_model(model_path)

import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

image = cv2.imread("/content/drive/MyDrive/emotions_classification2/eINTERFACE_2021_Image/val/Fear/myvideo_frames0_jpg.rf.61ae5cfe8cc3452f5b534d02cc5a56ec.jpg")#read the images from directory

image_fromarray = Image.fromarray(image, 'RGB')  #image is converted to array
resize_image = image_fromarray.resize((128, 128)) #coverted array is rsized 
expand_input = np.expand_dims(resize_image,axis=0)
input_data = np.array(expand_input)
input_data = input_data/255 #normalizing 

pred = loaded_model.predict(input_data)
result = pred.argmax()
result

2

In [ ]:
training_set.class_indices

{'Anger': 0,
 'Disgust': 1,
 'Fear': 2,
 'Happiness': 3,
 'Sadness': 4,
 'Surprise': 5}